In [0]:
from collections import Counter
import random

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import utils

In [4]:
# For E2E Dataset
trainset = pd.read_csv('/content/EAP - EAP.csv')
trainset = trainset.assign(clean=utils.replace_punctuation(trainset['tokenized_sents']))
vocab_to_int, int_to_vocab = utils.get_tokens(trainset['clean'])
as_tokens = trainset['clean'].apply(lambda x: [vocab_to_int[each] for each in x.split()])
trainset = trainset.assign(tokenized=as_tokens)
trainset.head()


,tokenized_sents,clean,tokenized
0,This process however afforded me no means of a...,This process however afforded me no means of a...,"[88, 2450, 84, 1028, 31, 37, 189, 4, 4155, 3, ..."
1,In his left hand was a gold snuff box from whi...,In his left hand was a gold snuff box from whi...,"[64, 22, 150, 158, 11, 7, 576, 2206, 338, 30, ..."
2,The astronomer perhaps at this point took refu...,The astronomer perhaps at this point took refu...,"[27, 6630, 250, 18, 25, 157, 195, 6631, 9, 3, ..."
3,The surcingle hung in ribands from my body,The surcingle hung in ribands from my body,"[27, 4156, 522, 9, 9307, 30, 14, 164]"
4,I knew that you could not say to yourself ster...,I knew that you could not say to yourself ster...,"[8, 311, 10, 40, 52, 19, 78, 6, 672, 6632, 121..."


In [10]:
from transformers import BertTokenizer

# Load the BERT tokenizer.
print('Loading BERT tokenizer...')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

Loading BERT tokenizer...


In [9]:
!pip install transformers

     |████████████████████████████████| 645kB 5.0MB/s 
     |████████████████████████████████| 3.8MB 21.6MB/s 
     |████████████████████████████████| 890kB 40.9MB/s 
     |████████████████████████████████| 1.1MB 38.0MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=91bb7b2ebfc73d1c3914ef204594b58fbb9d41a060fe85a7f8a02c0be028a370
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [11]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
max_len = 0

# For every sentence...
for sent in trainset['tokenized_sents']:

    # Tokenize the text and add `[CLS]` and `[SEP]` tokens.
    input_ids = tokenizer.encode(sent, add_special_tokens=True)

    # Update the maximum sentence length.
    max_len = max(max_len, len(input_ids))

print('Max sentence length: ', max_len)

Max sentence length:  298


In [15]:
# Tokenize all of the sentences and map the tokens to thier word IDs.
input_ids = []
attention_masks = []

# For every sentence...
for sent in trainset['tokenized_sents']:
    # `encode_plus` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    #   (5) Pad or truncate the sentence to `max_length`
    #   (6) Create attention masks for [PAD] tokens.
    encoded_dict = tokenizer.encode_plus(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = 300,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )
    
    # Add the encoded sentence to the list.    
    input_ids.append(encoded_dict['input_ids'])
    
    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks.append(encoded_dict['attention_mask'])

# Convert the lists into tensors.
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)


# Print sentence 0, now as a list of IDs.
print('Original: ',trainset['tokenized_sents'][0])
print('Token IDs:', input_ids[0])
print('Attention masks:',attention_masks[0])

Original:  This process however afforded me no means of ascertaining the dimensions of my dungeon as I might make its circuit and return to the point whence I set out without being aware of the fact so perfectly uniform seemed the wall
Token IDs: tensor([  101,  2023,  2832,  2174, 22891,  2033,  2053,  2965,  1997,  2004,
        17119, 18249,  2075,  1996,  9646,  1997,  2026, 16633,  2004,  1045,
         2453,  2191,  2049,  4984,  1998,  2709,  2000,  1996,  2391,  2043,
         3401,  1045,  2275,  2041,  2302,  2108,  5204,  1997,  1996,  2755,
         2061,  6669,  6375,  2790,  1996,  2813,   102,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,  